In [26]:
import pandas as pd
import datetime
import numpy as np

In [56]:
smiles_df = pd.read_excel('Polymer-SMILES-FP.xlsx').drop_duplicates('SMILES')

In [57]:
data = pd.read_excel('Melt Viscosity Data.xlsx', ["Homopolymer", "Copolymer", "Polymer Blend"])
data["Copolymer"] = data["Copolymer"].drop(columns = ["INDEX"])
#data['Copolymer'].loc[0, data["Copolymer"].columns.isin(fp_columns)]

In [58]:
#make new FP for CoPolymers
fp_columns = smiles_df.columns[3:]
for i in data["Homopolymer"].index:
    s_1 = data["Homopolymer"].loc[i,"SMILES"]
    if '*' not in str(data["Homopolymer"].loc[i,"SMILES"]):
        s_1 = smiles_df.loc[smiles_df["Polymer"] == data["Homopolymer"].loc[i,"Polymer"], "SMILES"].values
        s_1 = np.nan if len(s_1) == 0 else s_1[0]
        data["Homopolymer"].loc[i,"SMILES"] = s_1

for i in data["Copolymer"].index:
    s_1 = data["Copolymer"].loc[i,"SMILES 1"]
    if '*' not in str(data["Copolymer"].loc[i,"SMILES 1"]):
        s_1 = smiles_df.loc[smiles_df["Polymer"] == data["Copolymer"].loc[i,"Polymer 1"], "SMILES"].values
        s_1 = np.nan if len(s_1) == 0 else s_1[0]
        data["Copolymer"].loc[i,"SMILES 1"] = s_1
    s_2 = data["Copolymer"].loc[i,"SMILES 2"]
    if '*' not in str(data["Copolymer"].loc[i,"SMILES 2"]):
        s_2 = smiles_df.loc[smiles_df["Polymer"] == data["Copolymer"].loc[i,"Polymer 2"], "SMILES"].values
        s_2 = np.nan if len(s_2) == 0 else s_2[0]
        data["Copolymer"].loc[i,"SMILES 2"] = s_2
    w_1 = data["Copolymer"].loc[i,"Weight 1"]
    w_2 = data["Copolymer"].loc[i,"Weight 2"]
    fp_1 = smiles_df.loc[smiles_df["SMILES"] == s_1, list(fp_columns.values)]
    fp_2 = smiles_df.loc[smiles_df["SMILES"] == s_2, list(fp_columns.values)]
    fp = (fp_1*w_1).add(fp_2*w_2, fill_value = 0).sum()
    data["Copolymer"].loc[i, fp_columns] = fp

#Make new FP for Polymer Blends
for i in data["Polymer Blend"].index:
    s_1 = data["Polymer Blend"].loc[i,"SMILES 1"]
    if '*' not in str(data["Polymer Blend"].loc[i,"SMILES 1"]):
        s_1 = smiles_df.loc[smiles_df["Polymer"] == data["Polymer Blend"].loc[i,"Polymer 1"], "SMILES"].values
        s_1 = np.nan if len(s_1) == 0 else s_1[0]
        data["Polymer Blend"].loc[i,"SMILES 1"] = s_1
    s_2 = data["Polymer Blend"].loc[i,"SMILES 2"]
    if '*' not in str(data["Polymer Blend"].loc[i,"SMILES 2"]):
        s_2 = smiles_df.loc[smiles_df["Polymer"] == data["Polymer Blend"].loc[i,"Polymer 2"], "SMILES"].values
        s_2 = np.nan if len(s_2) == 0 else s_2[0]
        data["Polymer Blend"].loc[i,"SMILES 2"] = s_2
    w_1 = data["Polymer Blend"].loc[i,"Weight 1"]
    w_2 = data["Polymer Blend"].loc[i,"Weight 2"]
    fp_1 = smiles_df.loc[smiles_df["SMILES"] == s_1, list(fp_columns.values)]
    fp_2 = smiles_df.loc[smiles_df["SMILES"] == s_2, list(fp_columns.values)]
    fp_prod = (fp_1.reset_index(drop = True)*fp_2.reset_index(drop = True)).loc[0,:]
    fp = 1/((w_2*fp_1).add((w_1*fp_2), fill_value = 0).sum())
    fp = (fp_prod*fp).fillna(0)
    data["Polymer Blend"].loc[i, fp_columns] = fp

In [59]:
data["Copolymer"]["Polymer 1"] = (data["Copolymer"]["Polymer 1"]  + ", " + data["Copolymer"]["Polymer 2"])
data["Copolymer"]["SMILES 1"] = (data["Copolymer"]["SMILES 1"] + ", " + data["Copolymer"]["SMILES 2"])
data["Copolymer"] = data["Copolymer"].rename(columns = {"Polymer 1": "Polymer", "SMILES 1": "SMILES"}).drop(columns = ["Polymer 2","SMILES 2"]).reset_index(drop=True)
data["Copolymer"]

,Unnamed: 0,SAMPLE_ID,PID 1,PID 2,coPID,SMILES,Polymer,Weight 1,Weight 2,Mn,...,afp_C4_C2_N1,afp_C2_C4_H1,bfp_344,mfp_MQNs15,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210,bfp_482
0,0.0,NaN,NaN,NaN,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.9400,0.0600,80000.0,...,0.008571,0.008571,0.017143,0.008571,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,NaN,NaN,NaN,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.8700,0.1300,NaN,...,0.018571,0.018571,0.037143,0.018571,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,NaN,NaN,NaN,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.8100,0.1900,NaN,...,0.027143,0.027143,0.054286,0.027143,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,NaN,NaN,NaN,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.7600,0.2400,53000.0,...,0.034286,0.034286,0.068571,0.034286,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,NaN,NaN,NaN,NaN,"[*]CC([*])c1ccccc1, [*]CC([*])C#N","polystyrene, polyacrylonitrile",0.7200,0.2800,56000.0,...,0.040000,0.040000,0.080000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,NaN,NaN,P370314,P370137,NaN,NaN,"poly[oxy(difluoromethylene)], poly{oxy[1,1,2-t...",0.0500,0.9500,NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
250,NaN,NaN,P370314,P070055,NaN,NaN,"poly[oxy(difluoromethylene)], poly[oxy(1,1,2,2...",0.5900,0.4100,NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
251,NaN,NaN,P094606,P090377,NaN,NaN,"poly[oxy(2,2,3,3,4,4-hexafluoropentane-1,5-diy...",0.1007,0.8993,NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
252,NaN,NaN,P094606,P090377,NaN,NaN,"poly[oxy(2,2,3,3,4,4-hexafluoropentane-1,5-diy...",0.1800,0.8200,NaN,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
data["Polymer Blend"]["Polymer 1"] = (data["Polymer Blend"]["Polymer 1"]  + ", " + data["Polymer Blend"]["Polymer 2"])
data["Polymer Blend"]["SMILES 1"] = (data["Polymer Blend"]["SMILES 1"] + ", " + data["Polymer Blend"]["SMILES 2"])
data["Polymer Blend"] = data["Polymer Blend"].rename(columns = {"Polymer 1": "Polymer", "SMILES 1": "SMILES"}).drop(columns = ["Polymer 2","SMILES 2"]).reset_index(drop=True)
data["Polymer Blend"]["Mw"] = data["Polymer Blend"]["Mw 1"]*data["Polymer Blend"]["Weight 1"] + data["Polymer Blend"]["Mw 2"]*data["Polymer Blend"]["Weight 2"]
data["Polymer Blend"] = data["Polymer Blend"].drop(columns = ["Mw 1", "Mw 2"])
data["Polymer Blend"]

,Unnamed: 0,INDEX,SAMPLE_ID,PID 1,PID 2,SMILES,Polymer,Weight 1,Weight 2,Temperature,...,afp_C2_C4_H1,bfp_344,mfp_MQNs15,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210,bfp_482,Mw
0,0.0,NaN,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
1,1.0,NaN,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
2,2.0,NaN,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
3,3.0,NaN,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
4,4.0,NaN,NaN,NaN,NaN,"[*]CCCCCC(=O)N[*], [*]Nc1ccc(NC(=O)c2ccc(C([*]...","Poly(hexano-6-lactam), PPTA",0.975,0.025,260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37075.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,NaN,NaN,NaN,P090175,P070013,"[*]OC(C)CC([*])=O, [*]CCO[*]","poly(3-hydroxybutyric acid), Poly(ethylene oxide)",6.200,-5.200,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2458400.0
206,NaN,NaN,NaN,P090175,P070013,"[*]OC(C)CC([*])=O, [*]CCO[*]","poly(3-hydroxybutyric acid), Poly(ethylene oxide)",7.200,-6.200,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2840400.0
207,NaN,NaN,NaN,P090175,P070013,"[*]OC(C)CC([*])=O, [*]CCO[*]","poly(3-hydroxybutyric acid), Poly(ethylene oxide)",8.200,-7.200,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3222400.0
208,NaN,NaN,NaN,P090175,P070013,"[*]OC(C)CC([*])=O, [*]CCO[*]","poly(3-hydroxybutyric acid), Poly(ethylene oxide)",9.200,-8.200,180,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3604400.0


In [61]:
data["Homopolymer"] = data["Homopolymer"].merge(smiles_df, how = 'left',on = 'SMILES').rename(columns = {"Polymer_x": "Polymer"}).drop(columns = ["INDEX", "PID"])

In [62]:
out = pd.concat([data["Homopolymer"], data["Copolymer"], data["Polymer Blend"]], ignore_index = True)
out = out.rename(columns = {"Melt Viscosity (poise)":"Melt_Viscosity", "Shear Rate":"Shear_Rate"}).rename_axis("INDEX")
#out.loc[,"Mw"] = 

In [63]:
#out["Mw"] = out["Mw"].fillna(np.nan)
#print(out["Mw"].isnull())
#out.loc[out["Mw"].isnull()] =
out.loc[out["Mw"].isnull() & out["Mn"].notnull(), "Mw"] = out.loc[out["Mw"].isnull() & out["Mn"].notnull() , "Mn"]*2

In [64]:
out = out.dropna(subset=['SMILES'])

In [66]:
out.to_excel('Data/full_data_' + str(datetime.date.today()) +'.xlsx')

In [12]:
zero_shear_data = out.loc[out["Shear_Rate"] == 0,:]
shear_data = out.loc[out["Shear_Rate"] != 0,:]

In [13]:
zero_shear_data.to_excel('Data/zero_shear_data_' + str(datetime.date.today()) + '.xlsx')
shear_data.to_excel('Data/shear_data_'+ str(datetime.date.today()) +'.xlsx')

In [14]:
str(datetime.date.today())

'2021-12-28'

In [25]:
data['Homopolymer'].loc[750:]

,SAMPLE_ID,SMILES,Polymer,Mn,Mw,PDI,PDI (zw),Temperature,Shear Rate,Melt Viscosity (poise),...,afp_C4_C2_N1,afp_C2_C4_H1,bfp_344,mfp_MQNs15,bfp_46,bfp_47,bfp_55,bfp_79,bfp_210,bfp_482
750,NaN,NaN,"poly{[1,3-bis(4-fluorobenzoyl)benzene]-alt-[2-...",NaN,183000.0,NaN,NaN,335.0,595.662144,57796.928842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
751,NaN,NaN,"poly{[1,3-bis(4-fluorobenzoyl)benzene]-alt-[2-...",NaN,183000.0,NaN,NaN,335.0,1122.018454,45409.096110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
752,NaN,NaN,"poly{[1,3-bis(4-fluorobenzoyl)benzene]-alt-[2-...",NaN,183000.0,NaN,NaN,335.0,2073.321573,36467.396741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
753,NaN,NaN,"poly{[1,3-bis(4-fluorobenzoyl)benzene]-alt-[2-...",NaN,183000.0,NaN,NaN,335.0,3831.186850,28651.202697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
754,NaN,NaN,"poly{[1,3-bis(4-fluorobenzoyl)benzene]-alt-[2-...",NaN,183000.0,NaN,NaN,335.0,7216.611621,23009.371808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,NaN,NaN,"poly[(decane-1,10-diamine)-alt-(decanedioic ac...",NaN,13400.0,NaN,NaN,220.0,6.302559,11280.202365,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,NaN,NaN,"poly[(decane-1,10-diamine)-alt-(decanedioic ac...",NaN,13400.0,NaN,NaN,220.0,7.742637,10675.804149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1052,NaN,NaN,"poly[(decane-1,10-diamine)-alt-(decanedioic ac...",NaN,13400.0,NaN,NaN,220.0,9.779984,10000.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1053,NaN,NaN,"poly[(decane-1,10-diamine)-alt-(decanedioic ac...",NaN,13400.0,NaN,NaN,220.0,0.000000,21930.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
